# Introduction
---
_by Rohit Ashiwal <rohit.ashiwal265@gmail.com>_

A natural model for quantum computation is to treat the quantum computer as a coprocessor, similar to that used for GPUs, FPGAs, and other adjunct processors. The primary control logic runs classical code on a classical "host" computer. When appropriate and necessary, the host program can invoke a subroutine that runs on the adjunct processor. When the subroutine completes, the host program gets access to the subroutine's results.

Q# is a domain-specific programming language used for expressing quantum algorithms. It is used for writing subroutines that execute on an adjunct quantum processor, under the control of a classical host program and computer. Until quantum processors are widely available, Q# subroutines execute on a simulator.

## Type Model

Q# is a _strongly-typed_ language, such that careful use of these types can help the compiler to provide stron guarantees about Q# programs at compile time.

In order to provide the strongest guarantees possible, conversions between types in Q# must be explicit using calls to functions which experss that conversion. Upcasts to compatible types handles implicitly.

Q# provides both primitive type, which can be used directly, and a variety of ways to produce new types from other types. We describe each in the rest of this section.

### Primitive Types

The language provides several _primitive types_, from which other types can be constructed:

- `Int` representing signed integer
- `BigInt` signed integer of arbitrary size
- `Double` representing double-precision floating-point number
- `Bool` for boolean types
- `Qubit` to represent quantum bit or qubit
- `Pauli` representing one of the four single-qubit Pauli operator
- `Result` to contain result of measurement
- `Range` represents a sequence of integers
- `String` representing a sequence of unicode characters
- `Unit` is a type that allows only one value `()`

Example:

```cs
let one = 1;
let pi = 3.14;
```

By default variables are immutable, but we can declare mutable variables by using `mutable` keyword.

Example:

```cs
mutable x = 1;
set x = 2;
```

### Array Types

Array is a data structure, which can store a fixed-size collection of elements of the same data type. In Q#, we can create an array by using square bracket around the elements of an array.

Example:

```cs
let arr = [1, 2, 3, 4];
let even = arr[1..2..4];
```

> ⚠️ Warning \
\
The elements of an array cannot be changed after the array has been created. Update-and-reassign statements and/or copy-and-update expressions can be used to construct a modified array.

Alternatively, we can declare an array by using the `new` keyword.

Example:

```cs
let zero = new Int[13];
let qubits = new Qubit[10];
```

### Tuple Types

Tuple types hold different types within themselves. Such nesting is finite, however, as tuple types cannot under any circumstances contain themselves.

We have already seen a typle, `()`, which is an _empty_ tuple value.

Tuple instances are `immutable`. Q# does not provide a mechanism to change the contents of a tuple once created.

Example:

```cs
let tup = (3, false);
```

### Operation and Function Types

A Q# operation is a quantum subroutine. That is, it is a callable routine that contains quantum operations whereas a Q# function is a classical subroutine used within a quantum algorithm. Specifically, functions may not allocate or borrow qubits, not may they call operations. It is possible, however, to pass them operations or qubits for processing. Functions are thus entirely deterministic in the sense that calling them with the same arguments will always produce same result.

Together, operations and functions are called _callables_.

### User-Defined Types

We can define new types using the `newtype` statement. Let's create a complex datatype.

```cs
newtype Complex = (Double, Double);
```

This statement creates a new type with two _anonymous_ items of type `Double`. Aside from anonymous items, user define types also support named items as of Q# version 0.7 or higher.

```cs
newtype Complex = (Re: Double, Im: Double);
```

And now, we can use this newly created type just like any other type. We can pass it to a function, create different instances of it, etc.

```cs
function ComplexAddition(a: Complex, b: Complex) : Complex {
    return Complex(a::Re + b::Re, a::Im + b::Im);
}
```

## Statement and Other Constructs

### Comments

Comments begin with two forward slashes, `//`, and continue until the end of line. A comment may appear anywhere in a Q# source file.

### Namespaces

A namespace is a declarative region that provides a scope to the identifiers (the names of types, functions, variables, etc) inside it.

Every Q# operation, and user-defined type is defined within a namespace. Q# follows the sames rules for naming as other .NET languages. However, Q# does not support relative references to namespaces. Implies, `c.d` will not resolve to `a.b.c.d` if `a.b` is already opened.

`open` directive is used to include a namespace. We can also form aliases using `as` keyword

Example:

```cs
namespace NS {
    open Microsoft.Quantum.Intrinsic;
    open Microsoft.Quantum.Math as Math;
}
```

### Control Flow

#### For Loop

The `for` statement supports iteration over an integer range or over an array. The binding of the declared symbol(s) bound to each value in the range or array. In particular, it is possible to destruct e.g. array items for an iteration over an array upon assignment to the loop variable(s).

Example:

```cs
for (qubit in qubits) {  // qubits: Qubit[]
    H(qubit);
}

for (idx in 0 .. Length(qubits) - 1) {
    set results w/= idx <- (idx, M(qubits[idx]));  // update-and-reassign statement
}
```

The loop variable is bound at each enterance to the loop body, and unbound at the end of the body. In particular, the loop variable is not bound after the for loop is completed.

#### Repeat-Until-Success Loop

The `repeat` statment supports the quantum "repeat until success" pattern. The block is repeated `until` a certained condition _is_ met.

Example:

```cs
using (qubit = Qubit()) {
    repeat {
        H(qubit);
        let result = M(qubit);
        Message($"{result}");
    } until (result == Zero);
}
```

#### While Loop

Repeat-until-success patterns have a very quantum-specific connotation. They are widely used in particular classes of quantum algorithms -- hence the dedicated language construct in Q#. However, loops that break based on a condition and whose execution length is thus unknown at compile time need to be handled with particular care in a quantum runtime. Their use within functions on the other hand is unproblematic, since these only contain code that will be executed on conventional (non-quantum) hardware.

> ℹ️ Note\
\
Q# supports to use of while loops within functions only.

```cs
mutable (item, idx) = (-1, 0);

while (idx < Length(arr) and item < 0) {
    set item = arr[idx];
    set item += 1;
}
```
    

### Conditional Statement

Q# supports `if` statements for conditional execution. Also, nested conditionals are allowed.

```cs
if (result == One) {
    X(target);
}

if (i == 1) {
    X(target);
} elif (i == 2) {
    Y(target);
} else {
    Z(target);
}
```

### Qubit Management

> ℹ️ Note\
\
None of these statements are allowed within the body of a function. They are only valid within operations.

#### Clean Qubits

The `using` statement is used to acquire new qubits for use during a statement block. The qubits are guaranteed to be initialized to the computational `Zero` state. The qubits should be in the computational `Zero` state at the end of the statement block; simulators are encouraged to enforce this.

> ⚠️ Warning\
\
Target machines expect that qubits are in the |0Γƒ⌐ state immediately before delocation, so that they can be reused and offered to other `using` blocks for allocation. Whenever possible, use unitary operations to return any allocated qubits to |0Γƒ⌐. If need be, `Reset` operation can be used to ensure that the measured qubit is returned to |0Γƒ⌐. Such a measurement will destroy any entanglement with the remaining qubits and can thus impact the computation.

Example:

```cs
using (register = Qubit[8]) {
    // Do stuff ...
    ResetAll(register);
}
```

## Congratulations 🎉

You've completed the introduction to Q#! Now, you're ready to write really powerful quantum algorithms!